In [143]:
import deepdish as dd
import os
import h5py
import pandas as pd
import numpy as np

In [86]:
data_dir = '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/calc_SNR_next_gen/real_events'

## Events

In [163]:
events = ['GW190426', 'GW150914', 'GW200129']
pos_file_paths = {'input':{}, 'output':{}}
for event in events:
    event_file = [file for file in list(os.walk(os.path.join(data_dir, 'downloaded_samples')))[0][-1] if event in file][0]
    pos_file_paths['input'][event] = os.path.join(data_dir, 'downloaded_samples', event_file)
    pos_file_paths['output'][event] = os.path.join(data_dir, 'selected_samples', f'{event}_samples.h5')
pos_file_paths

{'input': {'GW190426': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/calc_SNR_next_gen/real_events/downloaded_samples/IGWN-GWTC2p1-v2-GW190426_190642_PEDataRelease_mixed_cosmo.h5',
  'GW150914': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/calc_SNR_next_gen/real_events/downloaded_samples/IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5',
  'GW200129': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/calc_SNR_next_gen/real_events/downloaded_samples/IGWN-GWTC3p0-v2-GW200129_065458_PEDataRelease_mixed_cosmo.h5'},
 'output': {'GW190426': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/calc_SNR_next_gen/real_events/selected_samples/GW190426_samples.h5',
  'GW150914': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/calc_SNR_next_gen/real_events/selected_samples/GW150914_samples.h5',
  'GW200129': '/home/divyajyoti/ACADEMIC/Projects/Cardiff_University/calc_SNR_next_gen/real_events/selected_samples/GW200129_samples.h5'}}

In [164]:
for event in events:
    with h5py.File(pos_file_paths['input'][event], 'r') as event_data:
        print(event)
        data = event_data['C01:Mixed']['posterior_samples'][:]
        data_dict = {name:data[name] for name in data.dtype.names}
        print(len(data_dict['mass_1']))
        dd.io.save(pos_file_paths['output'][event], data_dict)

GW190426
16781
GW150914
3337
GW200129
1993


In [165]:
test_data = dd.io.load(pos_file_paths['output']['GW150914'])
test_data.keys()

dict_keys(['a_1', 'a_2', 'beta', 'chi_eff', 'chi_eff_infinity_only_prec_avg', 'chi_p', 'chi_p_2spin', 'chi_p_infinity_only_prec_avg', 'chirp_mass', 'chirp_mass_source', 'comoving_distance', 'cos_iota', 'cos_theta_jn', 'cos_tilt_1', 'cos_tilt_1_infinity_only_prec_avg', 'cos_tilt_2', 'cos_tilt_2_infinity_only_prec_avg', 'dec', 'final_mass', 'final_mass_source', 'final_spin', 'geocent_time', 'inverted_mass_ratio', 'iota', 'log_likelihood', 'luminosity_distance', 'mass_1', 'mass_1_source', 'mass_2', 'mass_2_source', 'mass_ratio', 'peak_luminosity', 'phase', 'phi_1', 'phi_12', 'phi_2', 'phi_jl', 'psi', 'psi_J', 'ra', 'radiated_energy', 'redshift', 'spin_1x', 'spin_1y', 'spin_1z', 'spin_1z_infinity_only_prec_avg', 'spin_2x', 'spin_2y', 'spin_2z', 'spin_2z_infinity_only_prec_avg', 'symmetric_mass_ratio', 'theta_jn', 'tilt_1', 'tilt_1_infinity_only_prec_avg', 'tilt_2', 'tilt_2_infinity_only_prec_avg', 'total_mass', 'total_mass_source', 'viewing_angle'])

## GW170817

In [157]:
input_file = os.path.join(data_dir, 'downloaded_samples', 'GW170817_GWTC-1.hdf5')

In [158]:
param_names_map = {'costheta_jn': 'cos_theta_jn',
                   'luminosity_distance_Mpc': 'luminosity_distance',
                   'right_ascension': 'ra',
                   'declination': 'dec',
                   'm1_detector_frame_Msun': 'mass_1',
                   'm2_detector_frame_Msun': 'mass_2',
                   'lambda1': 'lambda_1',
                   'lambda2': 'lambda_2',
                   'spin1': 'a_1',
                   'spin2': 'a_2',
                   'costilt1': 'cos_tilt_1',
                   'costilt2': 'cos_tilt_2'}

In [159]:
spin_type = 'highSpin'
with h5py.File(input_file, 'r') as event_data:
    data = event_data[f'IMRPhenomPv2NRT_{spin_type}_posterior'][:]
    data_dict = {param_names_map[name]:data[name] for name in data.dtype.names}
    data_dict['spin_1z'] = data_dict['a_1']*data_dict['cos_tilt_1']
    data_dict['spin_2z'] = data_dict['a_2']*data_dict['cos_tilt_2']
    data_dict['theta_jn'] = np.arccos(data_dict['cos_theta_jn'])
    data_dict['psi'] = np.zeros_like(data_dict['dec'])
    data_dict['geocent_time'] = np.full_like(data_dict['dec'], 1187008882.4)
    print(len(data_dict['mass_1']))
    print(data_dict.keys())
    dd.io.save(os.path.join(data_dir, 'selected_samples', f'GW170817_{spin_type}_samples.h5'), data_dict)

4041
dict_keys(['cos_theta_jn', 'luminosity_distance', 'ra', 'dec', 'mass_1', 'mass_2', 'lambda_1', 'lambda_2', 'a_1', 'a_2', 'cos_tilt_1', 'cos_tilt_2', 'spin_1z', 'spin_2z', 'theta_jn', 'psi', 'geocent_time'])
